In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
import sys

# Add the parent directory to the Python path
script_dir = Path().resolve() # Get the current working directory
sys.path.append(str(script_dir.parent))
import src.config as config 

In [3]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/840852
2024-07-08 00:03:14,613 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-07-08 00:03:14,613 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
# create feature view (if it doesn't exist yet)
# This feature view only uses on feature group, so the query is trivial
try:
    # create feature view if it doesn't exist yet
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print('Feature view already existed. Skip creation.')


# get feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/840852/fs/836675/fv/time_series_hourly_feature_view/version/1


In [5]:
ts_data, _ = feature_view.training_data(
    description='Time-series hourly taxi rides',
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (52.60s) 
2024-07-08 01:28:51,242 WARNING: VersionWarning: Incremented version to `1`.



In [6]:
# sort by `pickup_location_id` and `pickup_hour`
ts_data.sort_values(by=['pickup_location_id', 'pickup_hour'], inplace=True)
ts_data

,pickup_hour,rides,pickup_location_id
1538787,2022-01-01 00:00:00+00:00,0,1
2081971,2022-01-01 01:00:00+00:00,0,1
4165882,2022-01-01 02:00:00+00:00,0,1
229884,2022-01-01 03:00:00+00:00,0,1
4832274,2022-01-01 04:00:00+00:00,1,1
...,...,...,...
3383324,2024-07-07 17:00:00+00:00,3,265
3383125,2024-07-07 18:00:00+00:00,3,265
3383036,2024-07-07 19:00:00+00:00,3,265
3384362,2024-07-07 20:00:00+00:00,7,265


In [7]:
from src.data import transform_ts_data_into_features_and_target

features, targets = transform_ts_data_into_features_and_target(
    ts_data,
    input_seq_len=24*28, # one month
    step_size=23,
)

features_and_target = features.copy()
features_and_target['target_rides_next_hour'] = targets

print(f'{features_and_target.shape=}')

100%|██████████| 265/265 [03:32<00:00,  1.25it/s]


features_and_target.shape=(219776, 675)
